## 0. Introduction
In this challenge, we used LightGBM (tree-based algorithm) to predict monthly emission factor for each power plant in Puerto Rico with a mean absolute percentage error of 21%. We think this is due to a lack of high quality data.

### Response: emission factor
Emission factor of a power plant is defined as the amount of NO2 around the plant per fuel combusted. 

`emission_factor` = `ColumnAmountNO2` / `fuel_combusted`

Using OMI data from NASA, we extracted the `ColumnAmountNO2` for each power plant and +/- 15 pixels from its surrounding (to be explained later). Since `ColumnAmountNO2` is recorded at different intervals throughout a month, we took the average reading for every month.

We also managed to collect monthly electricity generated `generated` for 22 power plants from [EIA](https://www.eia.gov/electricity/data/eia923/) in year 2018. Since data for 2019 is not available, we decided to use data from 2017 as a proxy. To calculate `fuel_combusted`, we multiplied `generated` with average the `heat_rate` for different energy source provided by [EIA](https://www.eia.gov/electricity/annual/html/epa_08_01.html). `heat_rate` is the amount of fuel combusted per Kilowatt of electricity generated.

### Predictors: aggregate functions of band 1 to band 7 from Sentinel-5P OFFL NO2
The input to LightGBM is mainly data from Sentinel-5P OFFL NO2. Using `rasterstats`, we are able to generate the reading of every band based on the respective power plant. We then take the `mean`, `max`, `min`, and `std` of each plant based on the month.

In [ ]:
import os
import glob 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime

import rasterio as rio
import folium
import tifffile

# Geospatial libraries that we will be using for this
import rasterio
import rasterstats
import folium
import geopandas as gpd

from tqdm.notebook import trange, tqdm

import warnings
warnings.simplefilter(action='ignore')

pd.options.display.max_columns = None

DATA_DIR = '../input/ds4g-environmental-insights-explorer/eie_data/'

## 1. Dataset
We used an external [dataset](https://disc.gsfc.nasa.gov/datasets/OMNO2_003/summary?keywords=OMI) `omi` from the NASA. According to several research papers, such as [de Foy et. al](https://www.sciencedirect.com/science/article/abs/pii/S1352231015301291) and [Kharol et. al](https://www.sciencedirect.com/science/article/abs/pii/S1352231015302569), OMI (Ozone Monitoring Instrument) is used to capture the actual NO2 level. 

The variable `ColumnAmountNO2` consists of [estimates](https://aura.gesdisc.eosdis.nasa.gov/data/Aura_OMI_Level2/OMNO2.003/doc/README.OMNO2.pdf) of NO2 in the total vertical column density and its uncertainty. We think this is a better measure of NO2 compared to `NO2_column_number_density` given in Sentinel-5P OFFL NO2 because of its popularity among research papers. 

We also performed a statistical test to show that NO2 levels around power plants with coal, oil, and gas as primary fuels are higher than other areas. Please refer to 1. [here](https://www.kaggle.com/wjshenggggg/team-sunway-ds4g-appendix).

In [ ]:
omi = pd.read_csv("../input/omiv1/OMI-Aura_L2-OMNO2.csv")

# drop na
omi = omi[~omi['ColumnAmountNO2'].isna()]

omi.reset_index(inplace=True, drop=True)

omi.head()

`power_plants` contains information on each plant and is provided.

In [ ]:
power_plants = pd.read_csv(os.path.join(DATA_DIR, 'gppd', 'gppd_120_pr.csv'))

power_plants.head()

Using `rasterstats`, we are able to generate the reading of every band from Sentinel-5P OFFL NO2 based on the respective power plant. `df_ras_gen` is the outcome after we take the `mean`, `max`, `min`, and `std` of each plant based on the month. Please refer to 2. [here](https://www.kaggle.com/wjshenggggg/team-sunway-ds4g-appendix) for more information on how `df_ras_gen` is constructed. Thanks to [Maxime](https://www.kaggle.com/maxlenormand/simplified-emission-for-each-plant-update-1) for sharing this approach.

Additionally, there is a `generated` column. It records the total amount of electricity generated by the respective plant in a given month. This [monthly data](https://www.eia.gov/electricity/data/eia923/) is made available by EIA. Unfortunately, we only have data for 22 out of 35 plants for the year 2018. We are less worried because the other 13 plants are all renewable energy plants. 

To extrapolate the amount of electricity generated in 2019, we used the data in 2017 as a proxy since the electricity generated in the first half of 2018 was low due to a hurricane.

In [ ]:
df_ras_gen = pd.read_csv('../input/df-ras-gen/df_ras_gen.csv')

df_ras_gen.head()

In [ ]:
COL = 'generated'

g = df_ras_gen.groupby(['system:index'])[COL].mean()

merged_df = pd.merge(g, power_plants, on='system:index')

non_renew_energy = ['Oil', 'Coal', 'Gas']

merged_df['non_renew'] = 0
merged_df.loc[merged_df['primary_fuel'].isin(non_renew_energy), 'non_renew'] = 1

keep = ['system:index', COL, 'non_renew']

no2_df = merged_df[keep]

no2_df['system:index'] = no2_df['system:index'].str[-5:]

# no2_df.head()

The plot below shows the average `generated` for each power plant. Red represents renewable energy while blue represents non-renewable energy power plants.

In [ ]:
plt.title('Average `{}` for each power plant'.format(COL))
barlist = plt.bar(no2_df['system:index'], no2_df[COL])

for idx, row in no2_df.iterrows():
    if row['non_renew'] == 0:
        barlist[idx].set_color('r')
        
plt.xticks(rotation=90)        
plt.show()

In [ ]:
unique_plants = df_ras_gen['system:index'].unique()

# remove the 13 plants that are not present in EIA data
power_plants = power_plants[power_plants['system:index'].isin(unique_plants)]

power_plants.reset_index(drop=True, inplace=True)

Extract `latitude` and `longtitude` from `power_plants`.

In [ ]:
def split_column_into_new_columns(dataframe,
                                  column_to_split,
                                  new_column_one,
                                  begin_column_one,
                                  end_column_one):
    '''Function provided by Paul,
    https://www.kaggle.com/paultimothymooney/overview-of-the-eie-analytics-challenge
    '''
    for i in range(0, len(dataframe)):
        dataframe.loc[i, new_column_one] = dataframe.loc[i, column_to_split][begin_column_one:end_column_one]
    return dataframe

power_plants = split_column_into_new_columns(power_plants,'.geo','latitude',50,66)

power_plants = split_column_into_new_columns(power_plants,'.geo','longitude',31,48)

power_plants['latitude'] = power_plants['latitude'].astype(float)

power_plants['longitude'] = power_plants['longitude'].astype(float)

We scale the `latitude` and `longitude` to pixels of 148 and 475 to caputure the `ColumnAmountNO2` data around each power plant. Since OMI captures `ColumnAmountNO2` reading at different location, we decide to attribute the `ColumnAmountNO2` reading +/- 15 pixels from a power plant to itself. 

We chose the `DEFINED_BOUNDS` to be 15 as it shows the highest level of NO2 in power plant areas compared to areas with no power plant.

In [ ]:
def scale_lat_to_img_idx(lat):
    lat = float(lat)
    lat_img_idx = (18.6 - lat) * 148 / (18.6 - 17.9)
    return int(lat_img_idx)

def scale_lon_to_img_idx(lon):
    lon = float(lon)
    lon_img_idx = (67.3 + lon) * 475 / (67.3 - 65.2)
    return int(lon_img_idx)

In [ ]:
power_plants['img_idx_lt'] = power_plants['latitude'].apply(scale_lat_to_img_idx)

power_plants['img_idx_lg'] = power_plants['longitude'].apply(scale_lon_to_img_idx)

omi['img_idx_lt'] = omi['Latitude'].apply(scale_lat_to_img_idx)

omi['img_idx_lg'] = omi['Longitude'].apply(scale_lon_to_img_idx)

# filter out of bounds
omi = omi[(omi['img_idx_lt'] >= 0) & (omi['img_idx_lt'] <= 148)]
omi = omi[(omi['img_idx_lg'] >= 0) & (omi['img_idx_lt'] <= 475)]

omi.reset_index(inplace=True, drop=True)

In [ ]:
DEFINED_BOUNDS = 15

dic = {}

for idx, row in power_plants.iterrows():
    dic[row['system:index']] = []
    
    lat, lon = row['img_idx_lt'], row['img_idx_lg']
    
    for _, omi_row in omi.iterrows():
        
        omi_lat, omi_lon = omi_row['img_idx_lt'], omi_row['img_idx_lg']
        
        lat_in_bounds = omi_lat - DEFINED_BOUNDS <= lat <= omi_lat + DEFINED_BOUNDS
        lon_in_bounds = omi_lon - DEFINED_BOUNDS <= lon <= omi_lon + DEFINED_BOUNDS
        
        # if the OMI reading of a given location is
        # +/- 15 pixels from a power plant,
        # assign the reading to the power plant.
        if lat_in_bounds and lon_in_bounds:
            d = {}
            d['time'] = omi_row['Time']
            d['ColumnAmountNO2'] = omi_row['ColumnAmountNO2']
            
            dic[row['system:index']].append(d)

We then take the mean of `ColumnAmountNO2` based on power plant and month. Unfortunately, some plants do not have any `ColumnAmountNO2` recorded for the given month.

In [ ]:
final = pd.DataFrame()

# construct a df for all the plants
# with the recorded ColumnAmountNO2 and time
for k in dic.keys():
    tmp = pd.DataFrame()
    tmp['system:index'] = [k] * len(dic[k])
    tmp = pd.concat([tmp, pd.DataFrame(dic[k])], axis=1)
    final = pd.concat([final, tmp], axis=0, ignore_index=True)

final['month'] = pd.to_datetime(final['time']).dt.month

g = final.groupby(['system:index', 'month'])['ColumnAmountNO2'].mean()

g = g.reset_index()

g.head()

Merge `ColumnAmountNO2` with `df_ras_gen`! We remove any entry that has `generated` of 0. We suspect this is due to insufficient information.

In [ ]:
df = pd.merge(df_ras_gen, g, on=['system:index', 'month'])

df = df.drop_duplicates()

df['generated'] = df['generated'].astype(np.float32)

# remove `generated` of 0
df = df[df['generated'] > 0.01]

df.head()

In [ ]:
monthly_elec = df.groupby('month')['generated'].sum()

plt.title('Total electricity generated')
plt.xlabel('Months')
plt.plot(monthly_elec)
plt.show()

We believe the dip is due to insufficient information by EIA.

## 2. Compute emission factor (EF)
So far, we have the amount of electricity generated by each power plant for each month. But as outlined in the challenge, we need the estimate the emission factor.

According to a [post](https://www.enveraconsulting.com/how-to-calculate-air-emissions/) by Envera Consulting, we can think of emission factor as the pounds of NOx per million cubic feet of fuel combusted. 

Using [data](https://www.eia.gov/electricity/annual/html/epa_08_01.html) provided by EIA, we can estimate fuel combusted based on the source. For instance, for 1 Kilowatthour of electricity produced, 10,481 Btu of coal has to be combusted. The conversion rate table is as follows. 

In [ ]:
heat_rate_dict = {
    'Coal': 10_481,
    'Oil': 11_095,
    'Gas': 7_821
}

The fact that [EIA considers](https://www.eia.gov/tools/faqs/faq.php?id=74&t=11) electricity generation from biomass, hydro, solar, and wind to be carbon neutral, we consider those plants as having 0 emission factor. Hence, they are removed from our dataframe.

In [ ]:
plant_fuel = power_plants[['system:index', 'primary_fuel']]

plant_fuel_dict = {}

for _, row in plant_fuel.iterrows():
    plant_fuel_dict[row['system:index']] = row['primary_fuel'] 

df['primary_fuel'] = df['system:index'].map(plant_fuel_dict)

df = df[df['primary_fuel'].isin(non_renew_energy)]

Assign the `heat_rate` to power plants. As defined above, emission factor is now equal to `ColumnAmountNO2` / `fuel_combusted`




In [ ]:
df['heat_rate'] = df['primary_fuel'].map(heat_rate_dict)

df['fuel_combusted'] = df['heat_rate'] * df['generated']

df['emission_factor'] = df['ColumnAmountNO2'] / (df['fuel_combusted'])

df.head()

We visualize the distribution of emission factors and the emission factors based on each month below.

In [ ]:
sns.distplot(df['emission_factor'])
plt.title('Distribution of emission factors')
plt.show()

We observe that the emission factor is right skew. We decide to clip the emission factor greater than 90th quantile and lesser than 10th quantile.

In [ ]:
top_quantile = df['emission_factor'].quantile(.9)
bottom_quantile = df['emission_factor'].quantile(.1)

df.loc[df['emission_factor'] > top_quantile, 'emission_factor'] = top_quantile
df.loc[df['emission_factor'] < bottom_quantile, 'emission_factor'] = bottom_quantile

Based on our methodology, the monthly emission factor is as follows.

In [ ]:
monthly_ef = df.groupby('month')['emission_factor'].sum()

plt.title('Total emission factors')
plt.xlabel('Months')
plt.plot(monthly_ef)
plt.show()

## 3. Model
We will use `lightgbm` to predict emission factors using data from Sentinel-5P OFFL NO2 after taking the monthly aggregate functions for each plant.

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
exclude = ['generated', 'primary_fuel', 'ColumnAmountNO2', 'heat_rate',
          'year', 'month',
           'fuel_combusted', 'emission_factor']

le = LabelEncoder()
df['system:index'] = le.fit_transform(df['system:index'])

cols = [c for c in df.columns if c not in exclude]

X_tr = df[cols]
y_tr = df['emission_factor']

Since emission factors vary a lot, we decided to use mean absolute percentage error (MAPE) to gauge the peformance of our model. 

In [ ]:
# Setting hyperparameters
lgb_params = {
    'learning_rate': 0.05,
    'metric': 'mape',
    'objective': 'regression',
}

In [ ]:
# Create train and validation set
X_train, X_test, y_train, y_test = train_test_split(X_tr, y_tr, test_size=0.2, random_state=42)

In [ ]:
trn_data = lgb.Dataset(X_train.values, label=y_train)
val_data = lgb.Dataset(X_test.values, label=y_test)

lgb_clf = lgb.train(lgb_params,
                    trn_data,
                    1000,
                    valid_sets=[trn_data, val_data],
                    early_stopping_rounds=200,
                    verbose_eval=200)

y_pred = lgb_clf.predict(X_test, 
                         num_iteration=lgb_clf.best_iteration)

In [ ]:
plt.title('Prediction vs Actual')
plt.plot(y_pred, 'r--', label='predict')
plt.plot(y_test.values, color='b', label='actual')
plt.legend(loc='upper right')
plt.show()

Our model has a MAPE of 21%, likely due to insufficient data. We expect our model to perform better with more data provided.

## 4. Analysis: feature importance
As seen below, other than `system:index`, statistics of `troposheric_NO2_column_density`, `cloud_fraction`, and `NO2_column_number_density` play important roles in predicting the `emission_factor`.

In [ ]:
feature_score = lgb_clf.feature_importance()

features_df = pd.DataFrame({'features': cols,
                            'importance': feature_score})

features_df = features_df.sort_values('importance', ascending=False)

plt.figure(figsize=(8,6))
sns.barplot(x="importance", y="features", 
            data=features_df)
plt.title('LightGBM Features')
plt.show()

## 5. Recommendation
### Scalability and accuracy
This `lightgbm` is certainly scalable. It does not fare well on accuracy due to limited data.

### Pros and cons of the model
* Pros: Cost-effective as it only requires satelite data. 
* Cons: The model assumes `ColumnAmountNO2` of +/- 15 pixels from a power plant belongs to its "emission"; it only accounts for power plants with primary fuel of oil, gas, and coal and assumes renewable energy plant does not contribute to emission factor; construction of the response variable `emission_factor` relies heavily on accurate data on electricity generated by a power plant and heat rate.

## 6. Conclusion
Thanks to Kaggle for organizing such an interesting competition! Although we did not have sufficient high quality data, we relished in using our creativity to explore various methods to model and predict the emission factor.